In [ ]:
# When running for the first time, run in terminal to set up the environment:
    # python -m venv env
    # Set-ExecutionPolicy -ExecutionPolicy Bypass -Scope Process
    # .\env\Scripts\activate
    # pip install -r requirements.txt

In [53]:
import os
import glob
import soundfile as sf
import torch
import pandas as pd
from audiobox_aesthetics.infer import initialize_predictor

In [54]:
def main():
    audio_dir = r"C:\GitHub\audio-aesthetics\assets"
    audio_files = glob.glob(os.path.join(audio_dir, "*.wav")) + glob.glob(os.path.join(audio_dir, "*.mp3"))
    
    predictor = initialize_predictor()
    
    batch_data = []
    for fpath in audio_files:
        data, sr = sf.read(fpath)
        audio_tensor = torch.tensor(data, dtype=torch.float32).T
        batch_data.append({"path": audio_tensor, "sample_rate": sr, "filename": fpath})
    
    # Forward pass over all files
    predictions = predictor.forward(batch_data)
    
    results = []
    for pred, info in zip(predictions, batch_data):
        ce_val = pred.get("CE", None)
        cu_val = pred.get("CU", None)
        pc_val = pred.get("PC", None)
        pq_val = pred.get("PQ", None)
        
        file_name = os.path.basename(info["filename"])
        results.append([
            file_name,
            ce_val,
            cu_val,
            pc_val,
            pq_val
        ])
    
    # Rename columns in the final DataFrame
    df_scores = pd.DataFrame(results, columns=[
        "File",
        "Content Enjoyment (CE)",
        "Content Usefulness (CU)",
        "Production Complexity (PC)",
        "Production Quality (PQ)"
    ])
    print(df_scores)
    df_scores.to_csv("results/aesthetics_results.csv", index=False)

    return df_scores  # <-- return it so it's accessible

if __name__ == "__main__":
    df_scores = main()


c:\GitHub\audio-aesthetics\env\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
c:\GitHub\audio-aesthetics\env\lib\site-packages\torch\nn\functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


                             File  Content Enjoyment (CE)  \
0                 lineare_a2a.wav                7.297026   
1                 lineare_ace.wav                5.886321   
2          lineare_fiorentini.wav                7.197011   
3            lineare_garofalo.wav                6.113775   
4   lineare_hai-assicurazione.wav                5.740927   
5            lineare_postepay.wav                6.809984   
6             lineare_seresto.wav                5.346477   
7              lineare_suzuki.wav                5.964487   
8            playlist_bernina.mp3                6.759501   
9            playlist_citroen.mp3                5.785450   
10           playlist_melinda.mp3                6.228623   
11   playlist_natural-trainer.mp3                6.344067   
12       playlist_parmareggio.mp3                6.677727   
13           playlist_repower.mp3                6.305100   
14          playlist_tenderly.mp3                5.872531   
15         playlist_valf

In [55]:
# Function to extract mode and brand
def extract_mode_brand(filename):
    base = os.path.splitext(filename)[0]  # remove extension
    parts = base.split('_')
    mode = parts[0] if len(parts) > 0 else None
    brand = parts[1] if len(parts) > 1 else None
    return mode, brand

# Apply extraction
df_scores[['mode', 'brand']] = df_scores['File'].apply(lambda x: pd.Series(extract_mode_brand(x)))

# Reorder columns: keep File first, then mode and brand, then all remaining columns
cols = ['File', 'mode', 'brand'] + [col for col in df_scores.columns if col not in ['File', 'mode', 'brand']]
df_scores = df_scores[cols]

# Display result
print(df_scores)

                             File      mode              brand  \
0                 lineare_a2a.wav   lineare                a2a   
1                 lineare_ace.wav   lineare                ace   
2          lineare_fiorentini.wav   lineare         fiorentini   
3            lineare_garofalo.wav   lineare           garofalo   
4   lineare_hai-assicurazione.wav   lineare  hai-assicurazione   
5            lineare_postepay.wav   lineare           postepay   
6             lineare_seresto.wav   lineare            seresto   
7              lineare_suzuki.wav   lineare             suzuki   
8            playlist_bernina.mp3  playlist            bernina   
9            playlist_citroen.mp3  playlist            citroen   
10           playlist_melinda.mp3  playlist            melinda   
11   playlist_natural-trainer.mp3  playlist    natural-trainer   
12       playlist_parmareggio.mp3  playlist        parmareggio   
13           playlist_repower.mp3  playlist            repower   
14        

In [56]:
# Define the allowed brands
allowed_brands = ['repower', 'citroen', 'bernina', 'tenderly']

# Apply the exclusion logic
df_scores['exclude_flag'] = (
    (df_scores['mode'] == 'playlist') & (~df_scores['brand'].isin(allowed_brands))
).astype(int)

# Reorder columns: File, mode, brand, exclude_flag, then the rest
cols = ['File', 'mode', 'brand', 'exclude_flag'] + [
    col for col in df_scores.columns if col not in ['File', 'mode', 'brand', 'exclude_flag']
]
df_scores = df_scores[cols]

# Display result
print(df_scores)


                             File      mode              brand  exclude_flag  \
0                 lineare_a2a.wav   lineare                a2a             0   
1                 lineare_ace.wav   lineare                ace             0   
2          lineare_fiorentini.wav   lineare         fiorentini             0   
3            lineare_garofalo.wav   lineare           garofalo             0   
4   lineare_hai-assicurazione.wav   lineare  hai-assicurazione             0   
5            lineare_postepay.wav   lineare           postepay             0   
6             lineare_seresto.wav   lineare            seresto             0   
7              lineare_suzuki.wav   lineare             suzuki             0   
8            playlist_bernina.mp3  playlist            bernina             0   
9            playlist_citroen.mp3  playlist            citroen             0   
10           playlist_melinda.mp3  playlist            melinda             1   
11   playlist_natural-trainer.mp3  playl

In [29]:
# Import the table
df_recall = pd.read_excel(r"C:\GitHub\audio-aesthetics\tables\table_recall.xlsx")

# Group by 'Attributes' and calculate ratios
df_recall_aggregated = df_recall.groupby('Attribute').agg({
    'unaided': lambda x: x.sum() / x.count(),
    'aided': lambda x: x.sum() / x.count()
}).reset_index()

df_recall_aggregated['Attribute'] = df_recall_aggregated['Attribute'].replace('hai assicurazione', 'hai-assicurazione')

print(df_recall_aggregated)

            Attribute   unaided     aided
0                 a2a  0.233766  0.298701
1                 ace  0.090909  0.142857
2             bernina  0.064935  0.142857
3             citroen  0.116883  0.168831
4          fiorentini  0.038961  0.194805
5            garofalo  0.129870  0.207792
6   hai-assicurazione  0.012987  0.012987
7             melinda  0.051948  0.116883
8     natural trainer  0.038961  0.051948
9         parmareggio  0.000000  0.038961
10           postepay  0.051948  0.051948
11            repower  0.025974  0.077922
12            seresto  0.077922  0.155844
13             suzuki  0.051948  0.090909
14           tenderly  0.000000  0.051948
15          valfrutta  0.000000  0.025974
16             yakult  0.025974  0.025974


In [34]:
# Subset df_scores where exclude_flag == 0
df = df_scores[df_scores['exclude_flag'] == 0].copy()

# Drop 'File' and 'exclude_flag' columns
df = df.drop(columns=['File', 'exclude_flag'])

# Display result
print(df)

        mode              brand  Content Enjoyment (CE)  \
0    lineare                a2a                7.297026   
1    lineare                ace                5.886321   
2    lineare         fiorentini                7.197011   
3    lineare           garofalo                6.113775   
4    lineare  hai-assicurazione                5.740927   
5    lineare           postepay                6.809984   
6    lineare            seresto                5.346477   
7    lineare             suzuki                5.964487   
8   playlist            bernina                6.759501   
9   playlist            citroen                5.785450   
13  playlist            repower                6.305100   
14  playlist           tenderly                5.872531   

    Content Usefulness (CU)  Production Complexity (PC)  \
0                  7.631387                    7.103478   
1                  6.453405                    4.109528   
2                  7.041038                    6.490301

In [35]:
# Create mapping dictionaries
aided_dict = df_recall_aggregated.set_index('Attribute')['aided'].to_dict()
unaided_dict = df_recall_aggregated.set_index('Attribute')['unaided'].to_dict()

# Create new columns by mapping
df['aided_recall'] = df['brand'].map(aided_dict)
df['unaided_recall'] = df['brand'].map(unaided_dict)

# Display result
print(df)

        mode              brand  Content Enjoyment (CE)  \
0    lineare                a2a                7.297026   
1    lineare                ace                5.886321   
2    lineare         fiorentini                7.197011   
3    lineare           garofalo                6.113775   
4    lineare  hai-assicurazione                5.740927   
5    lineare           postepay                6.809984   
6    lineare            seresto                5.346477   
7    lineare             suzuki                5.964487   
8   playlist            bernina                6.759501   
9   playlist            citroen                5.785450   
13  playlist            repower                6.305100   
14  playlist           tenderly                5.872531   

    Content Usefulness (CU)  Production Complexity (PC)  \
0                  7.631387                    7.103478   
1                  6.453405                    4.109528   
2                  7.041038                    6.490301

In [37]:
df = df.rename(columns={
    'Content Enjoyment (CE)': 'content-enjoyment',
    'Content Usefulness (CU)': 'content-usefulness',
    'Production Complexity (PC)': 'production-complexity',
    'Production Quality (PQ)': 'production-quality'
})

print(df)

        mode              brand  content-enjoyment  content-usefulness  \
0    lineare                a2a           7.297026            7.631387   
1    lineare                ace           5.886321            6.453405   
2    lineare         fiorentini           7.197011            7.041038   
3    lineare           garofalo           6.113775            4.880628   
4    lineare  hai-assicurazione           5.740927            6.343976   
5    lineare           postepay           6.809984            6.938411   
6    lineare            seresto           5.346477            4.627346   
7    lineare             suzuki           5.964487            4.844994   
8   playlist            bernina           6.759501            5.221873   
9   playlist            citroen           5.785450            6.332827   
13  playlist            repower           6.305100            5.676331   
14  playlist           tenderly           5.872531            4.677937   

    production-complexity  production

In [ ]:
import statsmodels.api as sm

df['is-playlist'] = (df['mode'] == 'playlist').astype(int)

# Define independent variables (X) and dependent variable (y)
X = df[['is-lineare', 'production-complexity']]
y = df['unaided_recall']

# Add constant to X for intercept
X = sm.add_constant(X)

# Fit linear regression model
model = sm.OLS(y, X).fit()

# Display regression results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:         unaided_recall   R-squared:                       0.189
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.049
Date:                Mon, 09 Jun 2025   Prob (F-statistic):              0.390
Time:                        11:53:27   Log-Likelihood:                 17.803
No. Observations:                  12   AIC:                            -29.61
Df Residuals:                       9   BIC:                            -28.15
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.03

c:\GitHub\audio-aesthetics\env\lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
